In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import re

In [3]:
from sklearn.preprocessing import Normalizer

In [4]:
from sklearn.metrics import pairwise_distances

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df=pd.read_csv("flipkart_com-ecommerce_sample.csv")

In [7]:
originalDF=df.copy()

In [8]:
dropCols=['uniq_id','crawl_timestamp','product_url','pid','image','is_FK_Advantage_product','product_rating','overall_rating']

In [9]:
df=df.drop_duplicates(subset='product_name')

In [10]:
df=df.drop(dropCols,axis=1)

In [11]:
df.isna().sum()

product_name                 0
product_category_tree        0
retail_price                56
discounted_price            56
description                  1
brand                     3762
product_specifications       8
dtype: int64

In [12]:
def removeUnwantedElements(strValue):
#     print(strValue)
    strValue=re.sub("[^a-zA-Z]"," ",strValue)
    strValue=strValue.split()
    newStr="";
    for eachStr in strValue:
        if(len(eachStr)>2):
            newStr=newStr+" "+eachStr
    return newStr

In [13]:
for col in df.columns:
    if(df[col].dtypes=='object'):
        df[col]=df[col].fillna("")
        df[col]=df[col].apply(lambda rowVal: removeUnwantedElements(rowVal))
    else:
        df[col]=df[col].fillna(0).astype("float")


In [14]:
objectCols=list(df.dtypes[df.dtypes=='object'].index)

In [15]:
objectCols

['product_name',
 'product_category_tree',
 'description',
 'brand',
 'product_specifications']

In [16]:
df['newCols']=""


In [17]:
for col in objectCols:
    df['newCols']=df['newCols']+" "+df[col]

In [18]:
df.newCols[0]

'  Alisha Solid Women Cycling Shorts  Clothing Women Clothing Lingerie Sleep Swimwear Shorts Alisha Shorts Alisha Solid Women Cycling Shorts  Key Features Alisha Solid Women Cycling Shorts Cotton Lycra Navy Red Navy Specifications Alisha Solid Women Cycling Shorts Shorts Details Number Contents Sales Package Pack Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women Fabric Care Gentle Machine Wash Lukewarm Water Not Bleach Additional Details Style Code ALTHT the Box shorts  Alisha  product specification key Number Contents Sales Package value Pack key Fabric value Cotton Lycra key Type value Cycling Shorts key Pattern value Solid key Ideal For value Women value Gentle Machine Wash Lukewarm Water Not Bleach key Style Code value ALTHT value shorts'

In [19]:
df.shape

(12676, 8)

## Doing TF IDF Vectorizer

In [20]:
vectorizer=TfidfVectorizer(analyzer="word",min_df=0.01,stop_words="english")

In [21]:
x=vectorizer.fit_transform(df.newCols)

In [22]:
x.shape

(12676, 653)

In [23]:
vocabulary=vectorizer.get_feature_names()

In [24]:
x=x.toarray()

In [25]:
x.shape

(12676, 653)

In [36]:
df_modified=pd.DataFrame(x)

In [39]:
df_modified['retail_price']=df['retail_price']
df_modified['discounted_price']=df['discounted_price']

In [41]:
df_modified.shape

(12676, 655)

In [42]:
df_modified=df_modified.fillna(0)

In [43]:
normalizer=Normalizer()

In [44]:
df_modified.shape

(12676, 655)

In [45]:
df_modified[['retail_price','discounted_price']]=normalizer.fit_transform(df_modified[['retail_price','discounted_price']])

In [47]:
df_modified.shape

(12676, 8)

In [48]:
result=pairwise_distances(df_modified,metric="cosine")

In [49]:
result.shape

(12676, 12676)

In [61]:
similarityDF=pd.DataFrame(result)

In [62]:
len(originalDF.pid.unique())

12675

In [63]:
originalDF=originalDF.drop_duplicates(subset='product_name')

In [64]:
similarityDF.columns=originalDF.pid

In [65]:
similarityDF.index=originalDF.pid

In [67]:
similarityDF.shape

(12676, 12676)

In [69]:
similarityDF.to_csv("recoFlipkartP1.csv")

In [9]:
x=pd.read_csv("recoFlipkartP1.csv")

In [10]:
x.head()

,pid,SRTEH2FF9KEDEFGF,SBEEH3QGU7MFYJFY,SHOEH4GRSUBJGZXE,PSOEH3ZYDMSYARJ5,PWTEB7H2E4KCYUE3,SHOEH3DZBFR88SCK,SHOEH4KM2W3Z6EH5,SWIEHF3EF5PZAZUY,PSOEH3ZYYFETGCCD,...,STIE5UVGW2JWVWCT,STIEC889ZGFD3RCE,STIE2KFZTQFVYZQQ,SHODYZ6SYRMHDYPB,STIE9F5UGVJFQYZH,SNDEY8UH5TZ2AJCK,STIE2ZEPACRQJKH7,SNDDX969ZZJJKSHB,SNDEF3R3VB55PJBX,STIE4NXGSXG5GFR2
0,SRTEH2FF9KEDEFGF,0.000000,0.475111,0.447381,0.949805,0.530671,0.499672,0.917077,0.863490,0.469085,...,0.466382,0.924805,0.942893,0.943626,0.924814,0.433184,0.963676,0.463135,0.892660,0.476473
1,SBEEH3QGU7MFYJFY,0.475111,0.000000,0.390794,0.916802,0.450887,0.428134,0.865517,0.887791,0.450230,...,0.445388,0.823518,0.893975,0.916958,0.845895,0.411119,0.930782,0.446141,0.880674,0.424897
2,SHOEH4GRSUBJGZXE,0.447381,0.390794,0.000000,0.912572,0.502820,0.303286,0.599169,0.893033,0.433188,...,0.425588,0.830945,0.900208,0.724128,0.840594,0.283615,0.936835,0.325357,0.642054,0.422807
3,PSOEH3ZYDMSYARJ5,0.949805,0.916802,0.912572,0.000000,0.974039,0.943017,0.876360,0.915530,0.333869,...,0.889218,0.771464,0.807372,0.891298,0.773893,0.860484,0.878643,0.935694,0.841662,0.933451
4,PWTEB7H2E4KCYUE3,0.530671,0.450887,0.502820,0.974039,0.000000,0.443906,0.962030,0.982797,0.504902,...,0.546738,0.963141,0.974533,0.976595,0.947536,0.494542,0.983139,0.499267,0.968464,0.528195


In [ ]:
2